# LSTM implementation for TORCS driver

In [82]:
import glob
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import Imputer
from collections import defaultdict

## LSTM Definition

In [2]:
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        super(RNN_LSTM, self).__init__()        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.out = nn.Linear(hidden_size, output_size)        
        self.hidden = self.init_hidden()
        
    def init_hidden(self, x=None):
        if x == None:
            return (Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)))
        else:
            return (Variable(x[0].data),Variable(x[1].data))
        
    def forward(self, x):
        lstm_out, self.hidden_out = self.lstm(x, self.hidden)
        output = self.out(lstm_out.view(len(x), -1))
        self.hidden = self.init_hidden(self.hidden_out)
        return output

## Train the network

The LSTM has 28 inputs, 3 hidden layers with 28 hidden units, and an output layer with 3 nodes. The batch size is set to 100, while the learning rate is variable along the 500 epochs.

In [66]:
INPUT_SIZE = 28
HIDDEN_SIZE = 28
NUM_LAYERS = 3
BATCH_SIZE = 100
NUM_EPOCHS = 500
# LRS = np.concatenate((np.arange(0.9, 0.1, -0.1),
#                       [0.15, 0.1, 0.05, 0.01, 0.0075, 0.005, 0.0025, 0.001, 0.00075, 0.0005, 0.00025, 0.0001]))
LRS = np.concatenate((np.arange(0.9, 0, -0.2),
                      [0.01, 0.005, 0.001, 0.0005, 0.0001]))

lstm_nn = RNN_LSTM(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, 3, BATCH_SIZE)
criterion = nn.MSELoss()

In [92]:
[normalize(i, 0, 1) for i in list((1, 2, 3, 4))]

[1.0, 2.0, 3.0, 4.0]

In [28]:
def normalize(x, min, max):
    '''Normalization of a vector to values between [0, 1]'''
    return (x - min)/(max-min)

# Get training filenames
training_files = glob.glob('')

# Search min and max values for specific parameters
maxSpeedX = -10000
minSpeedX = 10000
maxSpeedY = -10000
minSpeedY = 10000
maxRPM = 0
maxWheelSpin = 0

for f in training_files:
    train_ds = pd.read_csv(f)    
    X = train_ds.iloc[:, :-4].values
    
    if X[0].max() > maxSpeedX:
        maxSpeedX = X[0].max()
    if X[0].min() < minSpeedX:
        minSpeedX = X[0].min()
        
    if X[1].max() > maxSpeedY:
        maxSpeedY = X[1].max()
    if X[1].min() < minSpeedY:
        minSpeedY = X[1].min()    
    
    if X[4].max() > maxRPM:
        maxRPM = X[4].max()
    
    if X[5].max() > maxWheelSpin:
        maxWheelSpin = X[5].max()
    if X[6].max() > maxWheelSpin:
        maxWheelSpin = X[6].max()
    if X[7].max() > maxWheelSpin:
        maxWheelSpin = X[7].max()
    if X[8].max() > maxWheelSpin:
        maxWheelSpin = X[8].max()

# Save their values
param_dict = {
    'maxSpeedX':maxSpeedX,
    'minSpeedX':minSpeedX,
    'maxSpeedY':maxSpeedY,
    'minSpeedY':minSpeedY,
    'maxRPM':maxRPM,
    'maxWheelSpin':maxWheelSpin
}
with open('norm_parameters.pickle', 'wb') as handle:
    pickle.dump(param_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Read all training sets
training_sets = defaultdict(lambda: dict())

for f in training_files:
    # read dataset
    train_ds = pd.read_csv(f, header=False)
    
    X = train_ds.iloc[:, :-4].values
    y = train_ds.iloc[:, -4:].values
    
    # fill missing values with mean
    imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imputer = imputer.fit(X)
    X = imputer.transform(X)
    
    # normalize all values for interval [0, 1]    
    X[0] = normalize(X[0], minSpeedX, maxSpeedX)  # speedX = range(search min, search max)
    X[1] = normalize(X[1], minSpeedY, maxSpeedY)  # speedY = range(search min, search max)
    X[2] = normalize(X[2], -180, 180)  # angle = range(-180, 180)
    X[3] = normalize(X[3], -1, 6)  # currentGear = range(-1, 6)
    X[4] = normalize(X[4], 0, maxRPM) . # RPM = range(0, search max)
    for i in np.arange(5, 9):
        X[i] = normalize(X[i], 0, maxWheelSpin)  # *wheelSpin = range(0, search max)
    for i in np.arange(9, 28):
        X[i] = normalize(X[i], 0, 200)  # *sensorValues = range(0, 200)
    y[0] = normalize(y[0], -1, 6)  # gear = range(-1, 6)
    y[1] = normalize(y[1], -1, 1)  # steering = range(-1, 1)
    # for acceleration and break, compute their difference and normalize it
    accel_brake = y[2] - y[3]
    y[2] = normalize(accel_brake, -1, 1)  # accelerate-brake = range(-1, 1)
    y = np.delete(y, 3, axis=1)
    
    # Create TensorDataset from FloatTensors and save to dictionary
    X_train = torch.from_numpy(X).float()
    y_train = torch.from_numpy(y).float()
    dataset = TensorDataset(X_train, y_train)
    training_sets[f] = dataset

In [68]:
lridx = -1

for epoch in np.arange(NUM_EPOCHS):
    print('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    
    if epoch % 50 == 0:
        lridx += 1
        optimizer = torch.optim.Adam(rnn.parameters(), lr=LRS[lridx])
    
    for f in training_files:
#         print('  training set: %s' %(f[f.find('/')+1:]))        
        train_loader = DataLoader(dataset=training_sets[f], batch_size=BATCH_SIZE, shuffle=False)    
        lstm_nn.init_hidden()

        for i, (X, y) in enumerate(train_loader):
            if (len(X) != BATCH_SIZE):
                continue

            data = Variable(X.view(-1, 1, INPUT_SIZE))
            target = Variable(y)

            optimizer.zero_grad()
            prediction = lstm_nn(data)
            loss = criterion(prediction, target)
            loss.backward()
            optimizer.step()

            if (i+1) % BATCH_SIZE == 0:
                print('    step: [%d/%d], loss: %.4f'
                      %(i+1, len(training_sets[f].target_tensor)//BATCH_SIZE, loss.data[0]))

print('Training done')

0.9
0.7
0.5
0.3
0.1
0.01
0.005
0.001
0.0005
0.0001


## Save model parameters

Parameters will be used by the driver to get a command to the server.

In [ ]:
torch.save(rnn.state_dict(), 'rnn_params.pt')

In [ ]:
torch.save(rnn, 'whole_net.pt')

### Backup code

In [ ]:
# for file in training_files:
#     # Train the network for each training session
#     print('Training file: %s' %(file))
    
#     train_ds = pd.read_csv(file)
#     X_train = train_ds.iloc[:, 3:].values
#     y_train = train_ds.iloc[:, :3].values
    
#     imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
#     imputer = imputer.fit(X_train)
#     X_train = imputer.transform(X_train)
    
#     X_train = torch.from_numpy(X_train).float()
#     y_train = torch.from_numpy(y_train).float()
    
#     dataset = TensorDataset(X_train, y_train)
    
#     train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     rnn.init_hidden()
    
#     for epoch in range(NUM_EPOCHS):
#         for i, (X, y) in enumerate(train_loader):
#             if (len(X) != BATCH_SIZE):
#                 continue
            
#             data = Variable(X.view(-1, 1, INPUT_SIZE))
#             target = Variable(y)
            
#             optimizer.zero_grad()
#             prediction = rnn(data)
#             loss = criterion(prediction, target)
#             loss.backward()
#             optimizer.step()
            
#             if (i+1) % 30 == 0:
#                 print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
#                       %(epoch+1, NUM_EPOCHS, i+1, len(X_train)//BATCH_SIZE, loss.data[0]))